In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np 
import pandas as pd 
import os
import torch
from sklearn.model_selection import train_test_split
try:
    from sentence_transformers import SentenceTransformer, InputExample, losses, util
except:
    !pip install sentence_transformers
    from sentence_transformers import SentenceTransformer, InputExample, losses, util
from torch.utils.data import DataLoader
from tqdm import tqdm
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, fbeta_score, hamming_loss, jaccard_score, average_precision_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 950.6 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=39ee0a1ddf33d307a897e9b06f40dbcf21d3e411254a3a5785fd6002bc4d364c
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence_transformers


In [2]:
BASE = '../input/pretrained-transformers-for-fine-tuning/models'
path_models = os.listdir(BASE)

In [3]:
os.listdir('../input/')

['multilang-org-dataset',
 'name-similarity-transformer',
 'pretrained-transformers-for-fine-tuning']

In [4]:
df = pd.read_csv('../input/multilang-org-dataset/data.csv')

In [5]:
train, test = train_test_split(df,test_size=0.4, 
                                random_state=128,
                                stratify=df['is_dup'])

In [6]:
test.is_dup.value_counts()

0    58528
1    58528
Name: is_dup, dtype: int64

In [7]:
# Two lists of sentences
name_1 = [x[0] for x in test[['name_1']].values.tolist()[:5000]]
name_2 = [x[0] for x in test[['name_2']].values.tolist()[:5000]]

In [8]:
print(name_1[0], name_2[0])

S.R. International Йосс Еластомерс Б.В.


In [9]:
y_test = test.is_dup.values.tolist()[:5000]

In [10]:
import gc

In [11]:
def get_pred(model, name1, name2, not_my = True):
    torch.cuda.empty_cache()
    gc.collect()
    with torch.no_grad():
        if not_my:
            model1 = SentenceTransformer(os.path.join(BASE, model), device = 'cuda')
        else:
            model1 = SentenceTransformer(os.path.join('../input/name-similarity-transformer', 'trained_MiniLM-L12-v2'), device = 'cuda')
        embeddings1 = model1.encode(name_1, convert_to_tensor=True)
        embeddings2 = model1.encode(name_2, convert_to_tensor=True)
        cosine_scores = util.cos_sim(embeddings1, embeddings2)
        y_pred = []
        for i in range(len(cosine_scores)):
            y_pred.append(float(cosine_scores[i][i]))
        return y_pred

def evaluate(y_test, y_pred):
    y_test = np.asarray(y_test, dtype='float32')
    y_predf = [1 if x > 0.7 else 0 for x in y_pred]
    y_predf = np.asarray(y_predf, dtype='float32')
    score = f1_score(y_test, y_predf)
    rec = recall_score(y_test, y_predf)
    pre = precision_score(y_test, y_predf)
    ac = accuracy_score(y_test, y_predf)
    fb = fbeta_score(y_test, y_predf, beta=0.5)
    hm = hamming_loss(y_test, y_predf)
    js = jaccard_score(y_test, y_predf)
    ap = average_precision_score(y_test, y_predf)
    return ac, pre, rec, score, fb, hm, js, ap

In [12]:
mermodel = {}
y = get_pred(path_models[0], name_1, name_2, not_my = False)
metrics = evaluate(y_test, y)
mermodel['trained_MiniLM-L12-v2'] = metrics
y = 0
metrics = None

Batches:   0%|          | 0/157 [00:00<?, ?it/s]

Batches:   0%|          | 0/157 [00:00<?, ?it/s]

In [13]:
for i in range(len(path_models)):
    y = get_pred(path_models[i], name_1, name_2)
    metrics = evaluate(y_test, y)
    mermodel[path_models[i]] = metrics

Batches:   0%|          | 0/157 [00:00<?, ?it/s]

Batches:   0%|          | 0/157 [00:00<?, ?it/s]

Batches:   0%|          | 0/157 [00:00<?, ?it/s]

Batches:   0%|          | 0/157 [00:00<?, ?it/s]

Batches:   0%|          | 0/157 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/157 [00:00<?, ?it/s]

Batches:   0%|          | 0/157 [00:00<?, ?it/s]

Batches:   0%|          | 0/157 [00:00<?, ?it/s]

Batches:   0%|          | 0/157 [00:00<?, ?it/s]

Batches:   0%|          | 0/157 [00:00<?, ?it/s]

In [14]:
list(mermodel.items())[0][0]

'trained_MiniLM-L12-v2'

In [15]:
mo = list(mermodel.items())
titles = []
accuracy = []
precision = []
recall = []
f1 = []
fb = []
hm = []
js = []
ap = []
for i in range(len(mo)):
    titles.append(mo[i][0])
    accuracy.append(mo[i][1][0])
    precision.append(mo[i][1][1])
    recall.append(mo[i][1][2])
    f1.append(mo[i][1][3])
    fb.append(mo[i][1][4])
    hm.append(mo[i][1][5])
    js.append(mo[i][1][6])
    ap.append(mo[i][1][7])

In [16]:
df_met = pd.DataFrame(data = {'Model':titles, 'Accuracy': accuracy, 'Precision': precision, 'Recall':recall, 'F1-score':f1, 'F-beta':fb, 'Hamming loss':hm, 'Jaccard similarity':js, 'average precision':ap})

In [17]:
df_met

,Model,Accuracy,Precision,Recall,F1-score,F-beta,Hamming loss,Jaccard similarity,average precision
0,trained_MiniLM-L12-v2,0.9998,1.000000,0.999596,0.999798,0.999919,0.0002,0.999596,0.999796
1,sentence-transformers_paraphrase-multilingual-...,0.6196,0.957075,0.243036,0.387637,0.602844,0.3804,0.240415,0.607604
2,sentence-transformers_paraphrase-MiniLM-L3-v2,0.5628,0.977049,0.120307,0.214234,0.403029,0.4372,0.119968,0.553346
3,sentence-transformers_all-MiniLM-L12-v2,0.5624,0.970684,0.120307,0.214080,0.402159,0.4376,0.119871,0.552580
4,sentence-transformers_multi-qa-mpnet-base-dot-v1,0.5596,0.925697,0.120711,0.213571,0.396657,0.4404,0.119552,0.547341
5,sentence-transformers_distiluse-base-multiling...,0.6758,0.961207,0.360113,0.523935,0.720633,0.3242,0.354954,0.663143
6,sentence-transformers_all-mpnet-base-v2,0.5604,0.931889,0.121518,0.215000,0.399310,0.4396,0.120448,0.548441
7,sentence-transformers_distiluse-base-multiling...,0.6340,0.962804,0.271700,0.423804,0.638157,0.3660,0.268877,0.622393
8,sentence-transformers_multi-qa-distilbert-cos-v1,0.5610,0.949045,0.120307,0.213544,0.399143,0.4390,0.119535,0.549977
9,sentence-transformers_all-MiniLM-L6-v2,0.5630,0.980263,0.120307,0.214311,0.403466,0.4370,0.120016,0.553732


In [18]:
df_met.to_csv('metrics1.csv', index = False)